## Advanced SQL Analytics & Case-Based Queries

## Task 1: Multi-table Join Analysis

### 1. Import libraries, Connect and Load Database

In [8]:
import sqlite3
import pandas as pd

# Connect to Chinook database
conn = sqlite3.connect('chinook.db')

### 2. List each customer with total number of orders and total revenue

In [11]:
query1 = """
SELECT c.CustomerId,
       c.FirstName || ' ' || c.LastName AS CustomerName,
       c.Country,
       COUNT(i.InvoiceId) AS TotalOrders,
       SUM(i.Total) AS TotalRevenue
FROM customers c
JOIN invoices i ON c.CustomerId = i.CustomerId
GROUP BY c.CustomerId
ORDER BY TotalRevenue DESC;
"""

df1 = pd.read_sql_query(query1, conn)
df1.head()

,CustomerId,CustomerName,Country,TotalOrders,TotalRevenue
0,6,Helena Holý,Czech Republic,7,49.62
1,26,Richard Cunningham,USA,7,47.62
2,57,Luis Rojas,Chile,7,46.62
3,45,Ladislav Kovács,Hungary,7,45.62
4,46,Hugh O'Reilly,Ireland,7,45.62


### 3. Retrieve the top 5 highest spending customers with their country names

In [14]:
query2 = """
SELECT c.CustomerId,
       c.FirstName || ' ' || c.LastName AS CustomerName,
       c.Country,
       SUM(i.Total) AS TotalSpent
FROM customers c
JOIN invoices i ON c.CustomerId = i.CustomerId
GROUP BY c.CustomerId
ORDER BY TotalSpent DESC
LIMIT 5;
"""

df2 = pd.read_sql_query(query2, conn)
df2

,CustomerId,CustomerName,Country,TotalSpent
0,6,Helena Holý,Czech Republic,49.62
1,26,Richard Cunningham,USA,47.62
2,57,Luis Rojas,Chile,46.62
3,45,Ladislav Kovács,Hungary,45.62
4,46,Hugh O'Reilly,Ireland,45.62


### 4. List product categories (genres) and their total revenue

In [17]:
query3 = """
SELECT g.Name AS Genre,
       SUM(ii.UnitPrice * ii.Quantity) AS TotalRevenue
FROM genres g
JOIN tracks t ON g.GenreId = t.GenreId
JOIN invoice_items ii ON t.TrackId = ii.TrackId
GROUP BY g.Name
ORDER BY TotalRevenue DESC;
"""

df3 = pd.read_sql_query(query3, conn)
df3.head()

,Genre,TotalRevenue
0,Rock,826.65
1,Latin,382.14
2,Metal,261.36
3,Alternative & Punk,241.56
4,TV Shows,93.53


## Task 2: Window Functions

### 1. Rank customer orders by amount

In [21]:
query1 = """
SELECT 
    CustomerId,
    InvoiceId,
    Total AS OrderAmount,
    ROW_NUMBER() OVER (PARTITION BY CustomerId ORDER BY Total DESC) AS RowNum,
    RANK() OVER (PARTITION BY CustomerId ORDER BY Total DESC) AS Rank,
    DENSE_RANK() OVER (PARTITION BY CustomerId ORDER BY Total DESC) AS DenseRank
FROM invoices;
"""

df_ranked_orders = pd.read_sql_query(query1, conn)
df_ranked_orders.head()

,CustomerId,InvoiceId,OrderAmount,RowNum,Rank,DenseRank
0,1,327,13.86,1,1,1
1,1,382,8.91,2,2,2
2,1,143,5.94,3,3,3
3,1,98,3.98,4,4,4
4,1,121,3.96,5,5,5


### 2. Calculate running total of sales by month

In [25]:
query2 = """
SELECT 
    strftime('%Y-%m', InvoiceDate) AS Month,
    Total,
    SUM(Total) OVER (PARTITION BY strftime('%Y-%m', InvoiceDate)
                     ORDER BY InvoiceDate ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS RunningTotal
FROM invoices
ORDER BY InvoiceDate;
"""

df_running_total = pd.read_sql_query(query2, conn)
df_running_total.head()

,Month,Total,RunningTotal
0,2009-01,1.98,1.98
1,2009-01,3.96,5.94
2,2009-01,5.94,11.88
3,2009-01,8.91,20.79
4,2009-01,13.86,34.65


### 3. Rank top 3 products per category by revenue

In [32]:
query3 = """
SELECT *
FROM (
    SELECT
        g.Name AS Genre,
        t.Name AS TrackName,
        SUM(ii.UnitPrice * ii.Quantity) AS Revenue,
        RANK() OVER (
            PARTITION BY g.GenreId
            ORDER BY SUM(ii.UnitPrice * ii.Quantity) DESC
        ) AS GenreRank
    FROM invoice_items ii
    JOIN tracks t ON ii.TrackId = t.TrackId
    JOIN genres g ON t.GenreId = g.GenreId
    GROUP BY g.GenreId, t.TrackId
)
WHERE GenreRank <= 3;
"""

df_top_products = pd.read_sql_query(query3, conn)
df_top_products.head()

,Genre,TrackName,Revenue,GenreRank
0,Rock,Balls to the Wall,1.98,1
1,Rock,Inject The Venom,1.98,1
2,Rock,Snowballed,1.98,1
3,Rock,Overdose,1.98,1
4,Rock,Deuces Are Wild,1.98,1


## Task 3: Common Table Expressions (CTEs)

### 1. CTE for customers with average order value > 5

In [50]:
query1 = """
WITH CustomerAvgOrder AS (
    SELECT 
        CustomerId,
        AVG(Total) AS AvgOrderValue
    FROM invoices
    GROUP BY CustomerId
)
SELECT c.CustomerId, c.FirstName || ' ' || c.LastName AS Name, 
       c.Country, a.AvgOrderValue
FROM CustomerAvgOrder a
JOIN customers c ON a.CustomerId = c.CustomerId
WHERE a.AvgOrderValue > 5;
"""

df_cte_avg_order = pd.read_sql_query(query1, conn)
df_cte_avg_order.head()

,CustomerId,Name,Country,AvgOrderValue
0,1,Luís Gonçalves,Brazil,5.660000
1,2,Leonie Köhler,Germany,5.374286
2,3,François Tremblay,Canada,5.660000
3,4,Bjørn Hansen,Norway,5.660000
4,5,František Wichterlová,Czech Republic,5.802857


### 2. CTE to filter recent 3 months of orders, then aggregate revenue

In [56]:
query_max = "SELECT MAX(InvoiceDate) FROM invoices;"
max_date = pd.read_sql_query(query_max, conn).iloc[0, 0]
print("Latest Invoice Date in DB:", max_date)

Latest Invoice Date in DB: 2013-12-22 00:00:00


In [54]:
query2 = """
WITH RecentInvoices AS (
    SELECT *
    FROM invoices
    WHERE InvoiceDate >= '2013-09-22'
)
SELECT 
    strftime('%Y-%m', InvoiceDate) AS Month,
    SUM(Total) AS Revenue
FROM RecentInvoices
GROUP BY Month
ORDER BY Month;
"""

df_cte_recent_revenue = pd.read_sql_query(query2, conn)
df_cte_recent_revenue.head()

,Month,Revenue
0,2013-10,37.62
1,2013-11,49.62
2,2013-12,38.62


## Task 4: Business Scenario Query

### 1. Prepare a monthly product-wise sales trend

In [149]:
query_sales_trend = """
SELECT 
    t.Name AS Product,
    strftime('%Y-%m', i.InvoiceDate) AS Month,
    SUM(ii.Quantity) AS TotalQuantity
FROM invoice_items ii
JOIN invoices i ON ii.InvoiceId = i.InvoiceId
JOIN tracks t ON ii.TrackId = t.TrackId
GROUP BY t.TrackId, Month
ORDER BY t.Name, Month;
"""

df_trend = pd.read_sql_query(query_sales_trend, conn)
df_trend.head()

,Product,Month,TotalQuantity
0,"""?""",2012-08,1
1,#9 Dream,2010-03,1
2,'Round Midnight,2013-01,1
3,(Anesthesia) Pulling Teeth,2010-12,1
4,(White Man) In Hammersmith Palais,2011-03,1


### 2. Pivot the table (rows → products, columns → months)

In [74]:
df_pivot = df_trend.pivot_table(
    index="Product", 
    columns="Month", 
    values="TotalQuantity", 
    aggfunc="sum"
).fillna(0)

df_pivot.head()

Month,2009-01,2009-02,2009-03,2009-04,2009-05,2009-06,2009-07,2009-08,2009-09,2009-10,...,2013-03,2013-04,2013-05,2013-06,2013-07,2013-08,2013-09,2013-10,2013-11,2013-12
Product,,,,,,,,,,,,,,,,,,,,,
"""?""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
#9 Dream,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Round Midnight,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
(Anesthesia) Pulling Teeth,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
(White Man) In Hammersmith Palais,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [155]:
df_trend[df_trend['TotalQuantity'] > 0].head()

,Product,Month,TotalQuantity
0,"""?""",2012-08,1
1,#9 Dream,2010-03,1
2,'Round Midnight,2013-01,1
3,(Anesthesia) Pulling Teeth,2010-12,1
4,(White Man) In Hammersmith Palais,2011-03,1


In [163]:
df_trend.groupby('Month')['TotalQuantity'].sum().sort_values(ascending=False).head(10)

Month
2011-07    38
2011-05    38
2009-02    38
2011-08    38
2011-09    38
2011-10    38
2011-12    38
2012-01    38
2012-02    38
2012-03    38
Name: TotalQuantity, dtype: int64

### 3. Get the last 3 months (chronologically)

In [88]:
last_3_months = df_pivot.columns[-3:]  # Get last 3 month-columns
df_last3 = df_pivot[last_3_months]
df_last3.head()

Month,2013-10,2013-11,2013-12
Product,,,
"""?""",0.0,0.0,0.0
#9 Dream,0.0,0.0,0.0
'Round Midnight,0.0,0.0,0.0
(Anesthesia) Pulling Teeth,0.0,0.0,0.0
(White Man) In Hammersmith Palais,0.0,0.0,0.0


### 4. Identify products with strictly decreasing sales

In [122]:
def is_decreasing(row):
    return row['2013-10'] > row['2013-11'] > row['2013-12']

df_declining = df_last3[df_last3['2013-12'] < df_last3['2013-10']]
df_declining.head()

Month,index,Product,2013-10,2013-11,2013-12
127,127,B.Y.O.B.,1.0,0.0,0.0
131,131,Back Door Man,1.0,0.0,0.0
185,185,Black Moon Creeping,1.0,0.0,0.0
194,194,Blissed & Gone,1.0,0.0,0.0
239,239,Bullet With Butterfly Wings,1.0,0.0,0.0


In [129]:
def is_decreasing(row):
    return row['2013-10'] >= row['2013-11'] >= row['2013-12']
df_last3[['Product', '2013-10', '2013-11', '2013-12']].head(10)
df_decliners = df_last3[df_last3['2013-12'] < df_last3['2013-10']]
df_decliners

Month,index,Product,2013-10,2013-11,2013-12
127,127,B.Y.O.B.,1.0,0.0,0.0
131,131,Back Door Man,1.0,0.0,0.0
185,185,Black Moon Creeping,1.0,0.0,0.0
194,194,Blissed & Gone,1.0,0.0,0.0
239,239,Bullet With Butterfly Wings,1.0,0.0,0.0
241,241,Burden In My Hand,1.0,0.0,0.0
295,295,Cherub Rock,1.0,0.0,0.0
343,343,Corinna,1.0,0.0,0.0
417,417,Disarm,1.0,0.0,0.0
443,443,Don't Stand so Close to Me,1.0,0.0,0.0
